# Data Setup for LTR Demonstration

* Prerequisites: Product catalog and signals are indexed

This notebook reindexes the product catalog with each products' previous clickthrough queries stored in a new `previous_click_queries` field. For example, if our signals collection contains these click events:

```
type    query              doc_id
------  ---------------  --------
click   ipad                    1
click   ipad                    2
click   ipad2                   1
click   ipad                    1
click   ipad case               3
click   ipad case black         4
click   ipad black              4
click   ipad                    1
```

The resulting product catalog would look like (for these docs, omitting all existing fields for each product):

```
  id  previous_click_queries
----  -------------------------
   1  [ipad, ipad2, ipad, ipad]
   2  [ipad]
   3  [ipad case]
   4  [ipad case black, black]
```

This demonstrates the value of using previous user interactions as relevance factors in an LTR model. 

## Constants

In [12]:
COLLECTION_NAME = 'bestbuy'
SIGNALS_COLLECTION_NAME = 'bestbuy_signals'
ZKHOST = 'localhost:9983/lwfusion/4.0.0-SNAPSHOT/solr'
SOLR_URL = 'http://localhost:8983/solr'

## Update Solr Schema to use a custom similarity for `previous_click_queries`

The default BM25 similarity will cap term frequency's contribution asymptotically.  Additionally, BM25 normalizes by document length, which means that longer documents will be penalized. 

Repeated matches in the `previous_click_queries` field indicate product popularity for a given term (which we want to reward), so let's set `k` and `b` to remove the dampening effect.

In [14]:
import json
import requests

signal_field_type = {
    'name': 'text_signals',
    'class': 'solr.TextField',
    'indexAnalyzer': {
        'filters': [
            {
                'class': 'solr.StopFilterFactory',
                'ignoreCase': 'true',
                'words': 'stopwords.txt'
            },
            {
                'class': 'solr.LowerCaseFilterFactory'
            }
        ],
        'tokenizer': {
            'class': 'solr.StandardTokenizerFactory'
        }
    },
    'multiValued': True,
    'positionIncrementGap': '100',
    'queryAnalyzer': {
        'filters': [
            {'class': 'solr.StopFilterFactory',
             'ignoreCase': 'true',
             'words': 'stopwords.txt'
             },
            {
                'class': 'solr.SynonymGraphFilterFactory',
                'expand': 'true',
                'ignoreCase': 'true',
                'synonyms': 'synonyms.txt'
            },
            {
                'class': 'solr.LowerCaseFilterFactory'
            }
        ],
        'tokenizer': {
            'class': 'solr.StandardTokenizerFactory'
        }
    },
    'similarity': {
        'class': 'solr.BM25SimilarityFactory',
        'k1': '100000',
        'b': '0'
    }
}

requests.post('{}/{}/schema'.format(SOLR_URL, COLLECTION_NAME), headers={
    'Content-type': 'application/json'
}, data=json.dumps({
    'add-field-type': signal_field_type
}))

requests.post('{}/{}/schema'.format(SOLR_URL, COLLECTION_NAME), headers={
    'Content-type': 'application/json'
}, data=json.dumps({
    'add-field': {
        'name': 'previous_click_queries',
        'type': 'text_signals',
        'stored': True,
        'indexed': True,
        'multiValued': True
    }
}))

<Response [200]>

## Create Spark SQL views over Solr collections using Fusion's `spark-solr` connector

In [8]:
spark.read.load(format='solr',
                collection=COLLECTION_NAME,
                zkhost=ZKHOST,
                flatten_multivalued='false',
                request_handler='/select'  # This will preserve the value of multi-valued fields
).createOrReplaceTempView('bestbuy')

spark.read.load(format='solr',
                     collection='bestbuy_signals',
                     zkhost=ZKHOST,
                     flatten_multivalued='false',
                     request_handler='/select'  # This will preserve the value of multi-valued fields
).createOrReplaceTempView('bestbuy_signals')

## Find all queries that led to a click of a product

1. Create a temporary view called `previous_click_queries_by_doc_id` with `<product ID, [ query1, query2, query3, ... ]` tuples
2. Left join `previous_click_queries_by_doc_id` with `bestbuy` on `doc_id` and reindex `bestbuy collection`, which now includes `previous_click_queries` field

In [10]:
SQL = """
select 
    doc_id, collect_list(query_s) as previous_click_queries
from 
    bestbuy_signals
group by doc_id
"""
spark.sql(SQL).createOrReplaceTempView("previous_click_queries_by_doc_id")

In [11]:
SQL = """
select a.*, b.previous_click_queries
from bestbuy a left join previous_click_queries_by_doc_id b
on a.id = b.doc_id
"""
new_product_catalog_df = spark.sql(SQL)

In [ ]:
new_product_catalog_df.write.save(format='solr',
                      collection='bestbuy',
                      flatten_multivalued='false',
                      zkhost=zkhost,
                      commit_within='10000')